<a href="https://colab.research.google.com/github/jammy0903/-jettyCVE-2021-28164-/blob/main/AI_secHW_enhancingHW_try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import matplotlib.pyplot as plt

# CIFAR-10 데이터 로드 함수
def load_CIFAR10(root_dataset, batch_size, width=32, height=32):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # CIFAR-10 데이터셋의 정확한 평균과 표준편차 적용
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        transforms.RandomErasing(p=0.5)
    ])

    transform_test = transforms.Compose([
        transforms.Resize((width, height)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    # CIFAR-10 데이터셋 다운로드 및 로드
    train_cifar10 = datasets.CIFAR10(root=root_dataset, train=True, download=True, transform=transform_train)
    test_cifar10 = datasets.CIFAR10(root=root_dataset, train=False, download=True, transform=transform_test)

    train_loader = DataLoader(dataset=train_cifar10, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset=test_cifar10, batch_size=batch_size, shuffle=False, num_workers=4)

    nb_classes = len(test_cifar10.classes)

    return train_loader, test_loader, nb_classes

# ResNet-50 사전 학습된 모델 불러오기 함수
def get_resnet50_pretrained(num_classes):
    model = models.resnet50(pretrained=True)
    # CIFAR-10은 32x32 이미지이므로 첫 번째 레이어 수정
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    # 드롭아웃 추가 및 fc 레이어 수정
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.fc.in_features, num_classes)
    )
    return model

# 트레이닝 함수
def train(device, model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    return train_loss

# 검증 함수
def validate(device, model, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_acc = 0

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            output = model(x)
            loss = criterion(output, y)
            _, pred = torch.max(output, 1)

            test_loss += loss.item()
            test_acc += torch.sum(pred == y.data).item()

    test_loss /= len(test_loader)
    test_acc /= len(test_loader.dataset)
    return test_loss, test_acc

if __name__ == '__main__':
    # 장치 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 하이퍼파라미터 설정
    root_dataset = './cifar10'
    batch_size = 32  # 배치 사이즈 32로 설정
    learning_rate = 1e-3  # AdamW의 일반적인 학습률
    epochs = 50  # 에폭 수 50으로 설정
    weight_decay = 5e-4

    # CIFAR-10 데이터 로드
    train_loader, test_loader, nb_classes = load_CIFAR10(root_dataset, batch_size)

    # 모델 불러오기 (사전 학습된 ResNet-50)
    model = get_resnet50_pretrained(nb_classes).to(device)

    # 손실 함수와 옵티마이저 설정
    # 레이블 스무딩 적용
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Cosine Annealing 학습률 스케줄러 설정
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    # 학습 로그 저장용
    train_log = {
        'train_loss': [],
        'test_loss': [],
        'acc': []
    }

    # 학습 과정
    for epoch in range(epochs):
        train_loss = train(device, model, train_loader, criterion, optimizer)
        test_loss, test_acc = validate(device, model, test_loader, criterion)

        train_log['train_loss'].append(train_loss)
        train_log['test_loss'].append(test_loss)
        train_log['acc'].append(test_acc)

        # 학습률 스케줄러 업데이트
        scheduler.step()

        print(f'{epoch+1:5d}/{epochs:5d}: Train Loss [{train_loss:.4f}] / Test Loss [{test_loss:.4f}], Test Accuracy [{test_acc*100:.2f}%]')

    # 학습 결과 그래프 출력
    plt.figure()
    plt.title('Train Loss vs Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(train_log['train_loss'], label='Train')
    plt.plot(train_log['test_loss'], label='Test')
    plt.legend()
    plt.show()

    plt.figure()
    plt.title('Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(train_log['acc'], label='Test')
    plt.legend()
    plt.show()


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]


    1/   50: Train Loss [1.5766] / Test Loss [1.3383], Test Accuracy [63.81%]
    2/   50: Train Loss [1.2200] / Test Loss [1.0669], Test Accuracy [76.79%]
    3/   50: Train Loss [1.0969] / Test Loss [0.9902], Test Accuracy [79.74%]
    4/   50: Train Loss [1.0243] / Test Loss [0.9248], Test Accuracy [83.12%]
    5/   50: Train Loss [0.9681] / Test Loss [0.8838], Test Accuracy [84.50%]
    6/   50: Train Loss [0.9259] / Test Loss [0.8846], Test Accuracy [84.36%]
    7/   50: Train Loss [0.8960] / Test Loss [0.8299], Test Accuracy [86.44%]
    8/   50: Train Loss [0.8807] / Test Loss [0.8304], Test Accuracy [86.47%]
    9/   50: Train Loss [0.8471] / Test Loss [0.7851], Test Accuracy [88.42%]
   10/   50: Train Loss [0.8213] / Test Loss [0.7874], Test Accuracy [88.39%]
   11/   50: Train Loss [0.8030] / Test Loss [0.7700], Test Accuracy [89.26%]
   12/   50: Train Loss [0.7823] / Test Loss [0.7442], Test Accuracy [89.82%]
   13/   50: Train Loss [0.7691] / Test Loss [0.7200], Test Accu